<a href="https://colab.research.google.com/github/DomMcOyle/NLP-Assigments-22-23/blob/Assignment-2/Assignment2DP(G)R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Keywords**: Transformers, Question Answering, CoQA

## Deadlines

* **December 11**, 2022: deadline for having assignments graded by January 11, 2023
* **January 11**, 2023: deadline for half-point speed bonus per assignment
* **After January 11**, 2023: assignments are still accepted, but there will be no speed bonus

## Overview

### Problem

Question Answering (QA) on [CoQA](https://stanfordnlp.github.io/coqa/) dataset: a conversational QA dataset.

### Task

Given a question $Q$, a text passage $P$, the task is to generate the answer $A$.<br>
$\rightarrow A$ can be: (i) a free-form text or (ii) unanswerable;

**Note**: a question $Q$ can refer to previous dialogue turns. <br>
$\rightarrow$ dialogue history $H$ may be a valuable input to provide the correct answer $A$.

### Models

We are going to experiment with transformer-based models to define the following models:

1.  $A = f_\theta(Q, P)$

2. $A = f_\theta(Q, P, H)$

where $f_\theta$ is the transformer-based model we have to define with $\theta$ parameters.

## The CoQA dataset

<center>
    <img src="https://drive.google.com/uc?export=view&id=16vrgyfoV42Z2AQX0QY7LHTfrgektEKKh" width="750"/>
</center>

For detailed information about the dataset, feel free to check the original [paper](https://arxiv.org/pdf/1808.07042.pdf).



## Rationales

Each QA pair is paired with a rationale $R$: it is a text span extracted from the given text passage $P$. <br>
$\rightarrow$ $R$ is not a requested output, but it can be used as an additional information at training time!

## Dataset Statistics

* **127k** QA pairs.
* **8k** conversations.
* **7** diverse domains: Children's Stories, Literature, Mid/High School Exams, News, Wikipedia, Reddit, Science.
* Average conversation length: **15 turns** (i.e., QA pairs).
* Almost **half** of CoQA questions refer back to **conversational history**.
* Only **train** and **validation** sets are available.

## Dataset snippet

The dataset is stored in JSON format. Each dialogue is represented as follows:

```
{
    "source": "mctest",
    "id": "3dr23u6we5exclen4th8uq9rb42tel",
    "filename": "mc160.test.41",
    "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. 
    Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. [...]" % <-- $P$
    "questions": [
        {
            "input_text": "What color was Cotton?",   % <-- $Q_1$
            "turn_id": 1
        },
        {
            "input_text": "Where did she live?",
            "turn_id": 2
        },
        [...]
    ],
    "answers": [
        {
            "span_start": 59,   % <-- $R_1$ start index
            "spand_end": 93,    % <-- $R_1$ end index
            "span_text": "a little white kitten named Cotton",   % <-- $R_1$
            "input_text" "white",   % <-- $A_1$      
            "turn_id": 1
        },
        [...]
    ]
}
```

### Simplifications

Each dialogue also contains an additional field ```additional_answers```. For simplicity, we **ignore** this field and only consider one groundtruth answer $A$ and text rationale $R$.

CoQA only contains 1.3% of unanswerable questions. For simplicity, we **ignore** those QA pairs.

## [Task 1] Remove unaswerable QA pairs

Write your own script to remove unaswerable QA pairs from both train and validation sets.

## Dataset Download


In [1]:
import os
import urllib.request
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!")

In [2]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test')  # <-- Why test? See next slides for an answer!

coqa-train-v1.0.json: 49.0MB [00:06, 8.05MB/s]                            


Download completed!


coqa-dev-v1.0.json: 9.09MB [00:01, 6.97MB/s]                            

Download completed!


#### Data Inspection

Spend some time in checking accurately the dataset format and how to retrieve the tasks' inputs and outputs!

In [6]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import tensorflow as tf


def set_seed(SEED):
  random.seed(SEED) # if you're using random
  np.random.seed(SEED) # if you're using numpy
  torch.manual_seed(SEED) # torch.cuda.manual_seed_all(SEED) is not required
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  tf.random.set_seed(SEED) # setting the seed for tensorflow too
  os.environ['TF_DETERMINISTIC_OPS'] = '1'

def extract_data(split_dataset):
  """
  function extracting data from the list of dictionaries in the CoQA dataset
  :params:
    split_dataset: list of dictionaries from where to extract the pairs of question and passage and corresponding the answer
  """  
  XQA = [] # list that will contain pairs (P,Q)
  YQA = [] # list that will contain the Answers
  for d in split_dataset: # scan each document
    for i in range(len(d["questions"])): # scan each question
      if d["answers"][i]["span_end"]!=-1: # discard unanswerable questions
        single_example = [] # prepare the single example...
        single_example.append(d["questions"][i]["input_text"]) #... with the question ...
        single_example.append(d["story"]) # ...and the passage
        XQA.append(single_example) # and append it
        YQA.append(d["answers"][i]["input_text"]) # add the answer
  return XQA, YQA

## [Task 2] Train, Validation and Test splits

CoQA only provides a train and validation set since the test set is hidden for evaluation purposes.

We'll consider the provided validation set as a test set. <br>
$\rightarrow$ Write your own script to:
* Split the train data in train and validation splits (80% train and 20% val)
* Perform splits such that a dialogue appears in one split only! (i.e., split at dialogue level)
* Perform splitting using the following seed for reproducibility: 42

#### Reproducibility Memo

Check back tutorial 2 on how to fix a specific random seed for reproducibility!

In [7]:
seed = 42 
set_seed(seed)

with open('coqa/train.json') as f:
  # loading the training json
  train_json = json.load(f)

with open('coqa/test.json') as f:
  # loading the test json
  test_json = json.load(f)

# splitting training data
train_data, val_data = train_test_split(train_json["data"],
                                        train_size=0.8,
                                        shuffle=True,
                                        random_state=seed)
# extracting X as list of pairs [Passage,Question] and Y as a list of strings (Answers) 
XQA_train, YQA_train = extract_data(train_data)
XQA_val, YQA_val = extract_data(val_data)
XQA_test, YQA_test = extract_data(test_json["data"])
del(train_json)
del(test_json)

print("First training example:")
print(XQA_train[0:17])
print(YQA_train[0:17])
print("First validation example:")
print(XQA_val[0])
print(YQA_val[0])
print("First test example:")
print(XQA_test[0])
print(YQA_test[0])

First training example:
[['Where is this taking place?', 'TUNIS, Tunisia (CNN) -- Polls closed late Sunday in Tunisia, the torchbearer of the so-called Arab Spring, but voters will not see results of national elections until Tuesday, officials said. \n\nOn Sunday, long lines of voters snaked around schools-turned-polling-stations in Tunis\'s upscale Menzah neighborhood, some waiting for hours to cast a vote in the nation\'s first national elections since the country\'s independence in 1956. \n\n"It\'s a wonderful day. It\'s the first time we can choose our own representatives," said Walid Marrakchi, a civil engineer who waited more than two hours, and who brought along his 3-year-old son Ahmed so he could "get used to freedom and democracy." \n\nTunisia\'s election is the first since a popular uprising in January overthrew long-time dictator Zine El Abidine Ben Ali and triggered a wave of revolutions -- referred to as the Arab Spring -- across the region. \n\nMore than 60 political par

## [Task 3] Model definition

Write your own script to define the following transformer-based models from [huggingface](https://HuggingFace.co/).

* [M1] DistilRoBERTa (distilberta-base)
* [M2] BERTTiny (bert-tiny)

**Note**: Remember to install the ```transformers``` python package!

**Note**: We consider small transformer models for computational reasons!

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 58.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.2 MB/s 


In [9]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.8 MB/s 


In [32]:
# THIS IS A SEPARATOR ##########################################################

"""
This was tested with:
tensorflow==2.6
tensorflow-gpu==2.6
tensorflow-addons==0.16.1
transformers==4.18.0
Keras==2.6.0

Note 1: Simple adaptation of tf_seq2seq_lstm.py script
Note 2: make sure Keras and Tensorflow versions match!

"""

import tensorflow as tf
import tensorflow_addons as tfa
from tqdm import tqdm
from transformers import TFAutoModel, AutoTokenizer

# check if training can be performed on GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


class MyTrainer(object):
    """
    Simple wrapper class

    train_op -> uses tf.GradientTape to compute the loss
    batch_fit -> receives a batch and performs forward-backward passes (gradient included) 
    """

    def __init__(self, encoder, decoder, max_length):
        self.encoder = encoder
        self.decoder = decoder
        self.max_length = max_length
        self.ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                                reduction='none') # from logits means that it returns values after a 
                                                                                  # softmax application, thus it is useless to
                                                                                  # add a softmax activation layer if this parameter is set to 
                                                                                  # true (or even dangerous because it squashes the values)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-03)            # here it is possible to tweak the learning rate

    @tf.function
    def compute_loss(self, logits, target):
        loss = self.ce(y_true=target, y_pred=logits)
        mask = tf.logical_not(tf.math.equal(target, 0))
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask # pointwise product
        return tf.reduce_mean(loss)

    @tf.function
    def train_op(self, inputs):
        with tf.GradientTape() as tape:
            encoder_output, encoder_h, encoder_s = self.encoder({'input_ids': inputs['encoder_input_ids'],
                                                                 'attention_mask': inputs['encoder_attention_mask']})

            decoder_input = inputs['decoder_target'][:, :-1]  # ignore <end>
            real_target = inputs['decoder_target'][:, 1:]  # ignore <start>

            decoder.attention.setup_memory(encoder_output) # setup in order to perform attention queries over the 
                                                           # embedding space

            # decoder initialization, check build_initial_state for additional insights
            decoder_initial_state = self.decoder.build_initial_state(decoder.batch_size, [encoder_h, encoder_s])
            # the input is then passed to the initialized decoder and we obtain predictions
            # in rnn_output format because the model is BERT-emdedding-sequence-sequence, so the
            # last layer is still a sequence of cells (a RNN)
            predicted = self.decoder({'input_ids': decoder_input,
                                      'initial_state': decoder_initial_state}).rnn_output
            # we compute the losses over the computed predictions
            loss = self.compute_loss(logits=predicted, target=real_target)
        # gradients of the loss computed for this minibatch considering trainable
        # parameters of encoder and decoder
        grads = tape.gradient(loss, self.encoder.trainable_variables + self.decoder.trainable_variables)
        return loss, grads

    @tf.function
    def batch_fit(self, inputs):
        loss, grads = self.train_op(inputs=inputs)
        # applies gradients to the trainable variables using Adam
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_variables + self.decoder.trainable_variables))
        return loss

    # @tf.function
    def generate(self, input_ids, attention_mask=None):
        batch_size = input_ids.shape[0] # input_ids is the minibatch
        encoder_output, encoder_h, encoder_s = self.encoder({
            'input_ids': input_ids,
            'attention_mask': attention_mask
        })

        start_tokens = tf.fill([batch_size], output_tokenizer.word_index['<start>'])
        end_token = output_tokenizer.word_index['<end>']

        # samples the possible answer with greedy technique, we could possibly
        # use a variant here such as beam search at inference time 
        # We could not do this at training time, since the Sampler used at training
        # is not designed to project the token in an embedding space before computing
        # the next one. The aforementioned embedding space
        # is changing at each backpropagation step anyways, thus we stick with
        # the computation of the argmax of the logits using TrainingSampler.
        greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler() 
        # we have a decoder for training and a decoder for test time, thus
        # we need to re-define the training decoder each time we want to
        # train a new batch
        decoder_instance = tfa.seq2seq.BasicDecoder(cell=self.decoder.wrapped_decoder_cell,
                                                    sampler=greedy_sampler,
                                                    output_layer=self.decoder.generation_dense,
                                                    maximum_iterations=self.max_length)
        self.decoder.attention.setup_memory(encoder_output)

        # decoder_initial_state is still an output of the encoder, we pass it to
        # the decoder_instance in order to get the outputs
        decoder_initial_state = self.decoder.build_initial_state(batch_size, [encoder_h, encoder_s])
        decoder_embedding_matrix = self.decoder.embedding.variables[0]
        outputs, _, _ = decoder_instance(decoder_embedding_matrix,
                                         start_tokens=start_tokens,
                                         end_token=end_token,
                                         initial_state=decoder_initial_state)
        return outputs

    def translate(self, generated):
        return output_tokenizer.sequences_to_texts(generated.sample_id.numpy())


class Encoder(tf.keras.Model):

    def __init__(self, model_name, decoder_units):
        super(Encoder, self).__init__()
        self.model = TFAutoModel.from_pretrained(model_name, from_pt=True)
        self.reducer = tf.keras.layers.Dense(decoder_units)

    def call(self, inputs, training=False, **kwargs):
        model_output = self.model(inputs)
        all_outputs = model_output[0] # output of the last layer of the model
        pooled_output = model_output[1] # last layer but processed by a linear 
                                        # layer and a tanh
        pooled_output = self.reducer(pooled_output)
        return all_outputs, pooled_output, pooled_output


class Decoder(tf.keras.Model):

    def __init__(self, vocab_size, max_sequence_length, embedding_dim, decoder_units, batch_size):
        super(Decoder, self).__init__()

        self.max_sequence_length = max_sequence_length
        self.batch_size = batch_size

        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                   output_dim=embedding_dim)
        self.decoder_lstm_cell = tf.keras.layers.LSTMCell(self.decoder_units)
        # Just one type of attention, it could be changed to seek for different
        # results
        self.attention = tfa.seq2seq.BahdanauAttention(units=self.decoder_units,
                                                       memory=None,
                                                       memory_sequence_length=self.batch_size * [max_sequence_length])

        self.wrapped_decoder_cell = tfa.seq2seq.AttentionWrapper(self.decoder_lstm_cell,
                                                                 self.attention,
                                                                 attention_layer_size=self.decoder_units) # adds the attention mechanism after a single
                                                                                # LSTM cell, because we pass a word at the time
        # dense layer needed to generate the distribution values over 
        # the size of the vocabulary (probability for each word)
        self.generation_dense = tf.keras.layers.Dense(vocab_size)
        # Above we describe why this cannot be changed and why it resambles
        # the greedysampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        self.decoder = tfa.seq2seq.BasicDecoder(self.wrapped_decoder_cell,
                                                sampler=self.sampler,
                                                output_layer=self.generation_dense)

    def build_initial_state(self, batch_size, encoder_state):
        # after initializing the tensors within the attention layer to 0 we add
        # the designated initialization that allow us to query the embedding space,
        # which is passed as encoder_state.
        # We load the embedding of a single batch and we actually don't freeze 
        # the parameters related to BERT, that are modified and can possibly 
        # overfit. 
        initial_state = self.wrapped_decoder_cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)
        initial_state = initial_state.clone(cell_state=encoder_state) 
        return initial_state

    def call(self, inputs, training=False, **kwargs):
        # as shown in calls, inputs is a dictionary with entries: 
        # "input_ids" : _encoder_output_
        # "initial_state" : _result_of_build_initial_state_
        input_ids = inputs['input_ids']
        input_emb = self.embedding(input_ids)
        decoder_output, _, _ = self.decoder(input_emb,
                                            initial_state=inputs['initial_state'],
                                            sequence_length=self.batch_size * [self.max_sequence_length - 1])
        return decoder_output


def enconde_and_pad_sequence_input(sample_list, tokenizer):
  # sample_list is a list of strings. Each string represents a question, 
  # a passage or an answer
  encoded_inputs = tokenizer(sample_list, return_tensors='tf', padding=True)
  return encoded_inputs.input_ids, encoded_inputs.attention_mask


def encode_output_sequence(sample_list, tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<UNK>')):
  tokenizer.fit_on_texts(sample_list)
  encoded_output_sample = output_tokenizer.texts_to_sequences(output_sample)
  encoded_output_sample = tf.keras.preprocessing.sequence.pad_sequences(encoded_output_sample,
                                                                          padding='post',
                                                                          maxlen=max_sequence_length)
  return encoded_output_sample

def generate_batch(input_ids, attention_mask, encoded_output_sample, batch_index, 
                   batch_size):
    initial_index = batch_index * batch_size
    final_index = initial_index + batch_size

    batch = {
            'encoder_input_ids': input_ids[initial_index:final_index],
            'encoder_attention_mask': attention_mask[initial_index:final_index],
            'decoder_target': encoded_output_sample[initial_index:final_index]
    }
    return batch
    


if __name__ == '__main__':
    model_name = 'prajjwal1/bert-tiny'

    # tf.config.run_functions_eagerly(True) # needed for debugging, it linearizes
                                            # the computation graph 

    # Sample (list of lists, the inner lists are pairs (question, passage))
    input_sample = [
        ["How did you manage to escape from that cell?", """I was in jail. 
        I was convicted because I committed fraud, but luckily I still managed to send a letter to my beloved ones.
        The girl that I met before becoming a criminal, which would have become my future wife, gave me a help.
        She managed to hide a file in my birthday cake. Day after day I used it to destroy the window."""],
        ["Is this assignment hellish?", """For this assignment we require: basic coding knowledge,
        understanding of how encoder-decoder architectures work and a little bit of magic skills"""], 
        ["What color was the horse?", "Napoleon's horse was white"],
        ["What shape has the bowl?", "The bowl is red and round"],
        ["How did he escaped from the jail?", "Giovanni escaped from the cell using a smuggled file"]
    ]

    # list of answers
    output_sample = [
        "<start> My girlfriend hid a file in my birthday cake <end>",
        "<start> Just if you don't know how to design code <end>",
        "<start> white <end>", 
        "<start> round <end>", 
        "<start> with a file <end>"
    ]

    batch_size = 2

    # adding empty placeholders in case we don't have enough samples for the last
    # minibatch, but we're using parts of the network to learn something 
    # useless
    if len(input_sample) % batch_size != 0:
      n_remaining_samples = len(input_sample) % batch_size
      for i in range(0, n_remaining_samples):
        input_sample.append(["",""])
        output_sample.append("")
      print(input_sample)
      print(output_sample)

    # Input
    input_tokenizer = AutoTokenizer.from_pretrained(model_name)
    encoded_inputs = input_tokenizer(input_sample, return_tensors='tf', padding=True)
    input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
    max_input_length = input_ids.shape[-1]

    # Output
    output_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<UNK>')
    output_tokenizer.fit_on_texts(output_sample)

    output_vocab_size = len(output_tokenizer.word_index) + 1

    encoded_output_sample = output_tokenizer.texts_to_sequences(output_sample)
    max_output_length = max([len(item) for item in encoded_output_sample])

    max_sequence_length = max(max_input_length, max_output_length)

    encoded_output_sample = tf.keras.preprocessing.sequence.pad_sequences(encoded_output_sample,
                                                                          padding='post',
                                                                          maxlen=max_sequence_length)

    # Testing the encoder
    encoder = Encoder(model_name=model_name,
                      decoder_units=16)
    """
    encoder_output, encoder_h, encoder_s = encoder({'input_ids': input_ids,
                                                    'attention_mask': attention_mask})
    print(f'Printing the dimensions of the encoder: {encoder_output.shape} - {encoder_h.shape} - {encoder_s.shape}')
    """
    # Testing the decoder
    decoder = Decoder(vocab_size=output_vocab_size,
                      embedding_dim=50,
                      decoder_units=16,
                      batch_size=batch_size,
                      max_sequence_length=max_sequence_length)
    """
    decoder.attention.setup_memory(encoder_output)
    initial_state = decoder.build_initial_state(batch_size, [encoder_h, encoder_s])
    

    decoder_sample_batch = {
        'input_ids': tf.convert_to_tensor(encoded_output_sample, tf.int32),
        'initial_state': initial_state
    }
    sample_decoder_outputs = decoder(decoder_sample_batch).rnn_output
    print(f'Printing the dimensions of the decoder: {sample_decoder_outputs.shape}')
    """
    # Training
    trainer = MyTrainer(encoder=encoder,
                        decoder=decoder,
                        max_length=max_sequence_length)
    epochs = 100
    for epoch in tqdm(range(epochs)):
        """
        batch = {
            'encoder_input_ids': input_ids,
            'encoder_attention_mask': attention_mask,
            'decoder_target': encoded_output_sample
        }
        """
        batch_index = 0
        cumulative_loss = 0
        while batch_index * batch_size < len(input_ids):
          batch = generate_batch(input_ids, attention_mask, encoded_output_sample, batch_index, batch_size)
          loss = trainer.batch_fit(batch)
          cumulative_loss += loss
          print(f'Loss - {loss}')
          batch_index += 1
        mean_loss = cumulative_loss / batch_index
        print(f"Current mean {mean_loss}")
        # check results at the end of each epoch
        generated = trainer.generate(input_ids=input_ids,
                                     attention_mask=attention_mask)
        translated = trainer.translate(generated)
        print(f'Translated - {translated}')
        
        

# THIS IS A SEPARATOR ##########################################################

[['How did you manage to escape from that cell?', 'I was in jail. \n        I was convicted because I committed fraud, but luckily I still managed to send a letter to my beloved ones.\n        The girl that I met before becoming a criminal, which would have become my future wife, gave me a help.\n        She managed to hide a file in my birthday cake. Day after day I used it to destroy the window.'], ['Is this assignment hellish?', 'For this assignment we require: basic coding knowledge,\n        understanding of how encoder-decoder architectures work and a little bit of magic skills'], ['What color was the horse?', "Napoleon's horse was white"], ['What shape has the bowl?', 'The bowl is red and round'], ['How did he escaped from the jail?', 'Giovanni escaped from the cell using a smuggled file'], ['', '']]
['<start> My girlfriend hid a file in my birthday cake <end>', "<start> Just if you don't know how to design code <end>", '<start> white <end>', '<start> round <end>', '<start> with

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

Loss - 0.40622153878211975
Loss - 0.06946390867233276
Loss - 0.0815666913986206
Current mean 0.18575072288513184


  1%|          | 1/100 [00:13<22:36, 13.70s/it]

Translated - ['a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a', '<end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', '<end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>

...

Translated - ['my girlfriend hid a file in my birthday cake <end>', "just if you don't know how to design code <end>", 'white <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'round <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'with a file <end> <end> <end> <end> <end> <end> <end>', '<end> <end> <end> <end> <end> <end> <end> <end> <end> <end>']
Loss - 0.08156479150056839
Loss - 0.00025964234373532236
Loss - 0.006462354212999344
Current mean 0.029428929090499878


 99%|█████████▉| 99/100 [01:52<00:00,  1.30it/s]

Translated - ['my girlfriend hid a file in my birthday cake <end>', "just if you don't know how to <end> <end> <end>", 'white <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'round <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'with a file <end> <end> <end> <end> <end> <end> <end>', '<end> <end> <end> <end> <end> <end> <end> <end> <end> <end>']
Loss - 0.07925552129745483
Loss - 0.0002585137262940407
Loss - 0.006435692310333252
Current mean 0.028649909421801567


100%|██████████| 100/100 [01:52<00:00,  1.13s/it]

Translated - ['my girlfriend hid a file in my birthday cake <end>', "just if you don't know how to design <end> <end>", 'white <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'round <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'with a file <end> <end> <end> <end> <end> <end> <end>', '<end> <end> <end> <end> <end> <end> <end> <end> <end> <end>']


In [11]:
print(TFAutoModel.from_pretrained(model_name, from_pt=True))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
"""
This was tested with:
tensorflow==2.6
tensorflow-gpu==2.6
tensorflow-addons==0.16.1
transformers==4.18.0
Keras==2.6.0

Note 1: Simple adaptation of tf_seq2seq_lstm.py script
Note 2: make sure Keras and Tensorflow versions match!

"""

import tensorflow as tf
import tensorflow_addons as tfa
from tqdm import tqdm
from transformers import TFAutoModel, AutoTokenizer

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


class ModelTrainer(object):
    """
    Simple wrapper class

    train_op -> uses tf.GradientTape to compute the loss
    batch_fit -> receives a batch and performs forward-backward passes (gradient included)
    """

    def __init__(self, encoder, decoder, max_length):
        self.encoder = encoder
        self.decoder = decoder
        self.max_output_length = max_output_length
        self.ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-03)

    @tf.function
    def compute_loss(self, logits, target):
        loss = self.ce(y_true=target, y_pred=logits)
        mask = tf.logical_not(tf.math.equal(target, 0))
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_mean(loss)

    @tf.function
    def train_op(self, inputs):
        with tf.GradientTape() as tape:
            encoder_output, encoder_h, encoder_s = self.encoder({'input_ids': inputs['encoder_input_ids'],
                                                                 'attention_mask': inputs['encoder_attention_mask']})

            decoder_input = inputs['decoder_target'][:, :-1]  # ignore <end>
            real_target = inputs['decoder_target'][:, 1:]  # ignore <start>

            decoder.attention.setup_memory(encoder_output)

            decoder_initial_state = self.decoder.build_initial_state(decoder.batch_size, [encoder_h, encoder_s])
            predicted = self.decoder({'input_ids': decoder_input,
                                      'initial_state': decoder_initial_state}).rnn_output

            loss = self.compute_loss(logits=predicted, target=real_target)

        grads = tape.gradient(loss, self.encoder.trainable_variables + self.decoder.trainable_variables)
        return loss, grads

    @tf.function
    def batch_fit(self, inputs):
        loss, grads = self.train_op(inputs=inputs)
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_variables + self.decoder.trainable_variables))
        return loss

    # @tf.function
    def generate(self, input_ids, attention_mask=None):
        batch_size = input_ids.shape[0]
        encoder_output, encoder_h, encoder_s = self.encoder({
            'input_ids': input_ids,
            'attention_mask': attention_mask
        })

        start_tokens = tf.fill([batch_size], output_tokenizer.word_index['<start>'])
        end_token = output_tokenizer.word_index['<end>']

        greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
        decoder_instance = tfa.seq2seq.BasicDecoder(cell=self.decoder.wrapped_decoder_cell,
                                                    sampler=greedy_sampler,
                                                    output_layer=self.decoder.generation_dense,
                                                    maximum_iterations=self.max_output_length)
        self.decoder.attention.setup_memory(encoder_output)

        decoder_initial_state = self.decoder.build_initial_state(batch_size, [encoder_h, encoder_s])
        decoder_embedding_matrix = self.decoder.embedding.variables[0]
        outputs, _, _ = decoder_instance(decoder_embedding_matrix,
                                         start_tokens=start_tokens,
                                         end_token=end_token,
                                         initial_state=decoder_initial_state)
        return outputs

    def translate(self, generated):
        return output_tokenizer.sequences_to_texts(generated.sample_id.numpy())


class Encoder(tf.keras.Model):

    def __init__(self, model_name, decoder_units):
        super(Encoder, self).__init__()
        self.model = TFAutoModel.from_pretrained(model_name, from_pt=True)
        self.reducer = tf.keras.layers.Dense(decoder_units)

    def call(self, inputs, training=False, **kwargs):
        model_output = self.model(inputs)
        all_outputs = model_output[0]
        pooled_output = model_output[1]
        pooled_output = self.reducer(pooled_output)
        return all_outputs, pooled_output, pooled_output


class Decoder(tf.keras.Model):

    def __init__(self, vocab_size, max_sequence_length, embedding_dim, decoder_units, batch_size):
        super(Decoder, self).__init__()

        self.max_sequence_length = max_sequence_length
        self.batch_size = batch_size

        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                   output_dim=embedding_dim)
        self.decoder_lstm_cell = tf.keras.layers.LSTMCell(self.decoder_units)

        self.attention = tfa.seq2seq.BahdanauAttention(units=self.decoder_units,
                                                       memory=None,
                                                       memory_sequence_length=self.batch_size * [max_sequence_length])

        self.wrapped_decoder_cell = tfa.seq2seq.AttentionWrapper(self.decoder_lstm_cell,
                                                                 self.attention,
                                                                 attention_layer_size=self.decoder_units)

        self.generation_dense = tf.keras.layers.Dense(vocab_size)
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        self.decoder = tfa.seq2seq.BasicDecoder(self.wrapped_decoder_cell,
                                                sampler=self.sampler,
                                                output_layer=self.generation_dense)

    def build_initial_state(self, batch_size, encoder_state):
        initial_state = self.wrapped_decoder_cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)
        initial_state = initial_state.clone(cell_state=encoder_state)
        return initial_state

    def call(self, inputs, training=False, **kwargs):
        input_ids = inputs['input_ids']
        input_emb = self.embedding(input_ids)
        decoder_output, _, _ = self.decoder(input_emb,
                                            initial_state=inputs['initial_state'],
                                            sequence_length=self.batch_size * [self.max_sequence_length - 1])
        return decoder_output


if __name__ == '__main__':
    #model_name = 'distilroberta-base'
    model_name = 'prajjwal1/bert-tiny'

    # tf.config.run_functions_eagerly(True)

    # Sample
    batch_size = 16
    input_sample = XQA_train[:33]
    output_sample = [ "<start> " + ans + " <end>" for ans in YQA_train[:33]]
    print(output_sample)
#     [
#        "hello there how is it going",
#        "this assignment is hellish",
#        "how are you",
#        "what's your name Antonio?"
#    ]
#    [
#        "<start> it is going well <end>",
#        "<start> I agree <end>",
#        "<start> I'm fine, thanks <end>",
#        "<start> gnome <end>"
#    ]

    # batch_size = len(input_sample)

    # Input
    input_tokenizer = AutoTokenizer.from_pretrained(model_name)#, model_max_length=256)
    encoded_inputs = input_tokenizer(input_sample, return_tensors='tf', padding=True)#, truncation=True)
    input_ids, attention_mask = encoded_inputs.input_ids, encoded_inputs.attention_mask
    max_input_length = input_ids.shape[-1]

    # Output
    output_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<UNK>')
    output_tokenizer.fit_on_texts(output_sample)

    output_vocab_size = len(output_tokenizer.word_index) + 1

    encoded_output_sample = output_tokenizer.texts_to_sequences(output_sample)
    max_output_length = max([len(item) for item in encoded_output_sample])

    max_sequence_length = max(max_input_length, max_output_length)

    encoded_output_sample = tf.keras.preprocessing.sequence.pad_sequences(encoded_output_sample,
                                                                          padding='post',
                                                                          maxlen=max_sequence_length)

    # Test encoder
    encoder = Encoder(model_name=model_name,
                      decoder_units=16)
    encoder_output, encoder_h, encoder_s = encoder({'input_ids': input_ids,
                                                    'attention_mask': attention_mask})
    print(f'{encoder_output.shape} - {encoder_h.shape} - {encoder_s.shape}')

    # Test decoder
    decoder = Decoder(vocab_size=output_vocab_size,
                      embedding_dim=50,
                      decoder_units=16,
                      batch_size=batch_size,
                      max_sequence_length=max_sequence_length)
    

    # Training
    trainer = ModelTrainer(encoder=encoder,
                        decoder=decoder,
                        max_length=max_sequence_length)
    epochs = 100

    for epoch in tqdm(range(epochs)):
      for minibatch in range(int(len(input_sample)/batch_size)):
        batch = {
            'encoder_input_ids': input_ids[minibatch*batch_size:minibatch*batch_size+batch_size],
            'encoder_attention_mask': attention_mask[minibatch*batch_size:minibatch*batch_size+batch_size],
            'decoder_target': encoded_output_sample[minibatch*batch_size:minibatch*batch_size+batch_size]
        }
        decoder.attention.setup_memory(encoder_output[minibatch*batch_size:minibatch*batch_size+batch_size])
        initial_state = decoder.build_initial_state(batch_size,
                                                    [encoder_h[minibatch*batch_size:minibatch*batch_size+batch_size],
                                                     encoder_s[minibatch*batch_size:minibatch*batch_size+batch_size]])
        decoder_sample_batch = {
          'input_ids': tf.convert_to_tensor(encoded_output_sample[minibatch*batch_size:minibatch*batch_size+batch_size], tf.int32),
          'initial_state': initial_state
        }

        loss = trainer.batch_fit(batch)
        print(f'Loss - {loss}')

        generated = trainer.generate(input_ids=input_ids,
                                     attention_mask=attention_mask)
        translated = trainer.translate(generated)
        print(f'Translated - {translated}')



['<start> Tunisia <end>', '<start> Representatives  are being chosen <end>', '<start> Sunday <end>', '<start> 1956 <end>', '<start> Country gained its independence <end>', '<start> Menzah neighborhood <end>', '<start> Schools <end>', '<start> Yes <end>', '<start> Cnn <end>', '<start> yes <end>', '<start> George <end>', '<start> no <end>', '<start> yes <end>', '<start> Clarence <end>', '<start> no <end>', '<start> yes <end>', '<start> no <end>', '<start> no <end>', '<start> he lacked something one looked for in a lover <end>', '<start> yes <end>', '<start> financial <end>', '<start> yes <end>', '<start> Alexander Pop <end>', '<start> Tuesday <end>', '<start> Captain Putnam <end>', '<start> Run away <end>', '<start> Yes. <end>', '<start> No. <end>', '<start> Captain Putnam <end>', "<start> His father's <end>", '<start> Tom and Sam. <end>', '<start> "Glad you have come, Aleck." "I declare it looks as if you belonged here." <end>', '<start> "Remember the sport we used to have?" <end>']


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

(33, 413, 128) - (33, 16) - (33, 16)


  0%|          | 0/100 [00:00<?, ?it/s]

Loss - 0.02352244220674038
Translated - ['<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>', '<end>']
Loss - 0.03815056383609772


...

Translated - ['tunisia <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'representatives are being chosen <end> <end> <end> <end> <end> <end>', 'sunday <end> <end> <end> <end> <end> <end> <end> <end> <end>', '1956 <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'country gained its independence <end> <end> <end> <end> <end> <end>', 'menzah neighborhood <end> <end> <end> <end> <end> <end> <end> <end>', 'schools <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'cnn <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'george <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'no <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'clarence <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'no <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end>

 99%|█████████▉| 99/100 [06:03<00:03,  3.54s/it]

Translated - ['tunisia <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'representatives are being chosen <end> <end> <end> <end> <end> <end> <end>', 'sunday <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', '1956 <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'country gained its independence <end> <end> <end> <end> <end> <end> <end>', 'menzah neighborhood <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'schools <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'cnn <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'george <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'no <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'clarence <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>'

100%|██████████| 100/100 [06:07<00:00,  3.67s/it]

Translated - ['tunisia <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'representatives are being chosen <end> <end> <end> <end> <end> <end> <end> <end>', 'sunday <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', '1956 <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'country gained its independence <end> <end> <end> <end> <end> <end> <end> <end>', 'menzah neighborhood <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'schools <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'cnn <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'george <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'no <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> <end>', 'yes <end> <end> <end> <end> <end> <end> <end> <end> <end> <end> 

In [ ]:
len(input_ids)

1

## [Task 4] Question generation with text passage $P$ and question $Q$

We want to define $f_\theta(P, Q)$. 

Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$ and $Q_i$ and generate $A_i$.

## [Task 5] Question generation with text passage $P$, question $Q$ and dialogue history $H$

We want to define $f_\theta(P, Q, H)$. Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$, $Q_i$, and $H = \{ Q_0, A_0, \dots, Q_{i-1}, A_{i-1} \}$ to generate $A_i$.

## [Task 6] Train and evaluate $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$

Write your own script to train and evaluate your $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$ models.

### Instructions

* Perform multiple train/evaluation seed runs: [42, 2022, 1337].$^1$
* Evaluate your models with the following metrics: SQUAD F1-score.$^2$
* Fine-tune each transformer-based models for **3 epochs**.
* Report evaluation SQUAD F1-score computed on the validation and test sets.

$^1$ Remember what we said about code reproducibility in Tutorial 2!

$^2$ You can use ```allennlp``` python package for a quick implementation of SQUAD F1-score: ```from allennlp_models.rc.tools import squad```. 

## [Task 7] Error Analysis

Perform a simple and short error analysis as follows:
* Group dialogues by ```source``` and report the worst 5 model errors for each source (w.r.t. SQUAD F1-score).
* Inspect observed results and try to provide some comments (e.g., do the models make errors when faced with a particular question type?)$^1$

$^1$ Check the [paper](https://arxiv.org/pdf/1808.07042.pdf) for some valuable information about question/answer types (e.g., Table 6, Table 8) 

# Assignment Evaluation

The following assignment points will be awarded for each task as follows:

* Task 1, Pre-processing $\rightarrow$ 0.5 points.
* Task 2, Dataset Splitting $\rightarrow$ 0.5 points.
* Task 3 and 4, Models Definition $\rightarrow$ 1.0 points.
* Task 5 and 6, Models Training and Evaluation $\rightarrow$ 2.0 points.
* Task 7, Analysis $\rightarrow$ 1.0 points.
* Report $\rightarrow$ 1.0 points.

**Total** = 6 points <br>

We may award an additional 0.5 points for outstanding submissions. 
 
**Speed Bonus** = 0.5 extra points <br>

# Report

We apply the rules described in Assignment 1 regarding the report.
* Write a clear and concise report following the given overleaf template (**max 2 pages**).
* Report validation and test results in a table.$^1$
* **Avoid reporting** code snippets or copy-paste terminal outputs $\rightarrow$ **Provide a clean schema** of what you want to show

# Comments and Organization

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

Structure your code for readability and maintenance. If you work with Colab, use sections. 

This allows you to build clean and modular code, as well as easy to read and to debug (notebooks can be quite tricky time to time).

# FAQ (READ THIS!)

---

**Question**: Does Task 3 also include data tokenization and conversion step?

**Answer:** Yes! These steps are usually straightforward since ```transformers``` also offers a specific tokenizer for each model.

**Example**: 

```
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
encoded_text = tokenizer(text)
%% Alternatively
inputs = tokenizer.tokenize(text, add_special_tokens=True, max_length=min(max_length, 512))
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
```

**Suggestion**: Hugginface's documentation is full of tutorials and user-friendly APIs.

---
---

**Question**: I'm hitting **out of memory error** when training my models, do you have any suggestions?

**Answer**: Here are some common workarounds:

1. Try decreasing the mini-batch size
2. Try applying a different padding strategy (if you are applying padding): e.g. use quantiles instead of maximum sequence length

---
---

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# The End!

Questions?